# Reading Images

In [33]:
import pandas as pd
dataPath = r"C:\Users\HF\Desktop\Fall 2023\deep learning\menotme.csv"
data = pd.read_csv(dataPath)

In [34]:
data = data.sample(frac = 1)
data["Label"] = data["Label"].astype('category')
data["Label"] = data["Label"].cat.codes

In [35]:

import cv2
import numpy as np
resized=[]

for i in data["Path"]:
    image = cv2.imread(i,cv2.IMREAD_UNCHANGED)
    resize=cv2.resize(image,(220,220))#can resize however you want
    r = np.array(resize)
    resized.append(r)
    
    


C:\Users\HF\Desktop\Fall 2023\deep learning\iCloud Photos from Shaheen Amir\IMG_1369.JPEG
C:\Users\HF\Desktop\Fall 2023\deep learning\maryam\DMJX7482.JPG
C:\Users\HF\Desktop\Fall 2023\deep learning\images of not me\18_1_0_20170109214557098.jpg.chip.jpg
C:\Users\HF\Desktop\Fall 2023\deep learning\iCloud Photos from Shaheen Amir\IMG_5680.JPG
C:\Users\HF\Desktop\Fall 2023\deep learning\iCloud Photos from Shaheen Amir\IMG_4865.JPEG
C:\Users\HF\Desktop\Fall 2023\deep learning\images of not me\21_0_4_20161223214830361.jpg.chip.jpg
C:\Users\HF\Desktop\Fall 2023\deep learning\maryam\PMOY3226.JPG
C:\Users\HF\Desktop\Fall 2023\deep learning\images of not me\21_1_3_20170104222105822.jpg.chip.jpg
C:\Users\HF\Desktop\Fall 2023\deep learning\iCloud Photos from Shaheen Amir\IMG_5721.JPEG
C:\Users\HF\Desktop\Fall 2023\deep learning\maryam\IMG_7153.JPG
C:\Users\HF\Desktop\Fall 2023\deep learning\iCloud Photos from Shaheen Amir\IMG_0452.JPEG
C:\Users\HF\Desktop\Fall 2023\deep learning\maryam\DLOF5000.JP

In [36]:
y=data['Label']

In [37]:
resized=np.array(resized)

In [38]:
resized.shape

(153, 220, 220, 3)

In [39]:
from sklearn.preprocessing import LabelBinarizer
label_binarizer = LabelBinarizer()
y_one_hot_encoded = label_binarizer.fit_transform(y)

In [40]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(resized, y_one_hot_encoded, test_size=0.2)


In [41]:
import pickle
with open('train_test_splits.pkl', 'wb') as file:
    pickle.dump((X_train, X_test, y_train, y_test), file)
 

In [193]:
#for loading
import pickle
with open('train_test_splits.pkl', 'rb') as file:
    X_train1, X_test1, y_train1, y_test1 = pickle.load(file)

In [194]:
X_train1=X_train1/255
X_test1=X_test1/255

# Using NN

In [195]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model

image_height, image_width, num_channels = 220, 220, 3 

num_classes = 3 

input_shape = (image_height, image_width, num_channels)

inputs = Input(shape=input_shape)

x = Flatten()(inputs)
# x = Dense(256, activation='relu')(x)
x = Dense(128, activation='relu')(x)
x = Dense(64, activation='relu')(x)

outputs = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=inputs, outputs=outputs)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()


Model: "model_23"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 220, 220, 3)]     0         
                                                                 
 flatten_23 (Flatten)        (None, 145200)            0         
                                                                 
 dense_67 (Dense)            (None, 128)               18585728  
                                                                 
 dense_68 (Dense)            (None, 64)                8256      
                                                                 
 dense_69 (Dense)            (None, 3)                 195       
                                                                 
Total params: 18594179 (70.93 MB)
Trainable params: 18594179 (70.93 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [196]:
model.fit(X_train1, y_train1, epochs=10, batch_size=32,validation_split=0.2)


Epoch 1/10
4/4 [==============================] - 3s 441ms/step - loss: 65.9939 - accuracy: 0.3402 - val_loss: 32.6545 - val_accuracy: 0.3200
Epoch 2/10
4/4 [==============================] - 1s 271ms/step - loss: 31.4542 - accuracy: 0.3505 - val_loss: 15.3403 - val_accuracy: 0.2400
Epoch 3/10
4/4 [==============================] - 1s 283ms/step - loss: 13.5936 - accuracy: 0.4742 - val_loss: 12.6526 - val_accuracy: 0.3600
Epoch 4/10
4/4 [==============================] - 1s 269ms/step - loss: 5.2463 - accuracy: 0.5773 - val_loss: 10.1747 - val_accuracy: 0.6400
Epoch 5/10
4/4 [==============================] - 1s 273ms/step - loss: 14.2760 - accuracy: 0.5567 - val_loss: 3.2099 - val_accuracy: 0.3600
Epoch 6/10
4/4 [==============================] - 1s 266ms/step - loss: 6.6382 - accuracy: 0.5876 - val_loss: 25.0891 - val_accuracy: 0.3600
Epoch 7/10
4/4 [==============================] - 1s 283ms/step - loss: 14.8786 - accuracy: 0.5155 - val_loss: 3.4133 - val_accuracy: 0.3600
Epoch 8/10

In [197]:
predictions=model.predict(X_test1)

1/1 [==============================] - 0s 104ms/step


In [198]:
import numpy as np
predicted_labels = np.argmax(predictions, axis=1)

In [199]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(np.argmax(y_test1, axis=1), predicted_labels)

In [200]:
accuracy

0.7419354838709677

# Using CNN

In [224]:

from tensorflow.keras.layers import Conv2D, MaxPooling2D

x = Conv2D(32, (3, 3), activation='relu')(inputs)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Conv2D(128, (3, 3), activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

x = Flatten()(x)


x = Dense(128, activation='relu')(x)
x = Dense(64, activation='relu')(x)
outputs = Dense(num_classes, activation='softmax')(x) 

model1 = Model(inputs=inputs, outputs=outputs)

model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    'model_checkpoint.h5', monitor='', save_best_only=True, mode='min')
checkpoint_callback2=tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=2,
    verbose=0,
    mode="auto",
    baseline=None,
    restore_best_weights=True,
    start_from_epoch=0,
)
model1.summary()




Model: "model_27"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 220, 220, 3)]     0         
                                                                 
 conv2d_57 (Conv2D)          (None, 218, 218, 32)      896       
                                                                 
 max_pooling2d_57 (MaxPooli  (None, 109, 109, 32)      0         
 ng2D)                                                           
                                                                 
 conv2d_58 (Conv2D)          (None, 107, 107, 64)      18496     
                                                                 
 max_pooling2d_58 (MaxPooli  (None, 53, 53, 64)        0         
 ng2D)                                                           
                                                                 
 conv2d_59 (Conv2D)          (None, 51, 51, 128)       738

In [225]:
model1.fit(X_train1, y_train1, epochs=20, batch_size=32, validation_split=0.2,callbacks=[checkpoint_callback,checkpoint_callback2])


Epoch 1/20
4/4 [==============================] - ETA: 0s - loss: 1.7864 - accuracy: 0.3093WARNING:tensorflow:Can save best model only with  available, skipping.


4/4 [==============================] - 9s 2s/step - loss: 1.7864 - accuracy: 0.3093 - val_loss: 1.0727 - val_accuracy: 0.3200
Epoch 2/20
4/4 [==============================] - ETA: 0s - loss: 1.0520 - accuracy: 0.3402WARNING:tensorflow:Can save best model only with  available, skipping.


4/4 [==============================] - 7s 2s/step - loss: 1.0520 - accuracy: 0.3402 - val_loss: 1.0356 - val_accuracy: 0.7200
Epoch 3/20
4/4 [==============================] - ETA: 0s - loss: 0.9929 - accuracy: 0.7629WARNING:tensorflow:Can save best model only with  available, skipping.


4/4 [==============================] - 7s 2s/step - loss: 0.9929 - accuracy: 0.7629 - val_loss: 1.1136 - val_accuracy: 0.4400
Epoch 4/20
4/4 [==============================] - ETA: 0s - loss: 1.0123 - accuracy: 0.5155WARNING:tensorflow:Can save best model only with  available, skipping.


4/4 [==============================] - 7s 2s/step - loss: 1.0123 - accuracy: 0.5155 - val_loss: 0.7626 - val_accuracy: 0.8000
Epoch 5/20
4/4 [==============================] - ETA: 0s - loss: 0.6237 - accuracy: 0.8454WARNING:tensorflow:Can save best model only with  available, skipping.


4/4 [==============================] - 7s 2s/step - loss: 0.6237 - accuracy: 0.8454 - val_loss: 0.5588 - val_accuracy: 0.7600
Epoch 6/20
4/4 [==============================] - ETA: 0s - loss: 0.4795 - accuracy: 0.7732WARNING:tensorflow:Can save best model only with  available, skipping.


4/4 [==============================] - 9s 2s/step - loss: 0.4795 - accuracy: 0.7732 - val_loss: 0.4884 - val_accuracy: 0.8000
Epoch 7/20
4/4 [==============================] - ETA: 0s - loss: 0.3522 - accuracy: 0.8454WARNING:tensorflow:Can save best model only with  available, skipping.


4/4 [==============================] - 9s 2s/step - loss: 0.3522 - accuracy: 0.8454 - val_loss: 0.9968 - val_accuracy: 0.6400
Epoch 8/20
4/4 [==============================] - ETA: 0s - loss: 0.8368 - accuracy: 0.7216WARNING:tensorflow:Can save best model only with  available, skipping.


4/4 [==============================] - 7s 2s/step - loss: 0.8368 - accuracy: 0.7216 - val_loss: 1.1636 - val_accuracy: 0.5200


In [226]:
prediction2=model1.predict(X_test1)

1/1 [==============================] - 1s 502ms/step


In [227]:
predicted_labels2 = np.argmax(prediction2, axis=1)

In [228]:
accuracy2= accuracy_score(np.argmax(y_test1, axis=1), predicted_labels2)

In [229]:
accuracy2

0.8064516129032258